In [ ]:
#create a function to pull in data from dropbox using an API token
#import all necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt




In [ ]:
#get .env variables
from dotenv import load_dotenv
load_dotenv()

import os
from pathlib import Path
from dotenv import find_dotenv
import requests

#set up the path to the .env file
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

#get api token from .env file
link = os.getenv("api_box")

In [2]:
!pip install dropbox


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\joelw\Anaconda3\python.exe -m pip install --upgrade pip


In [1]:
#use dropbox API to pull in data

import dropbox
import pandas as pd
import io

dbx = dropbox.Dropbox(link)

def get_data_from_dropbox(file_path):
    md, res = dbx.files_download(file_path)
    df = pd.read_csv(io.BytesIO(res.content))
    return df

get_data_from_dropbox('/Apps/OBDLink/CsvLogs/WVGEF9BP9GD001746')

ModuleNotFoundError: No module named 'dropbox'